In [1]:
# Set number of threads to use
import os
nthreads = 12
os.environ["MKL_NUM_THREADS"] = str(nthreads)
os.environ["NUMEXPR_NUM_THREADS"] = str(nthreads)
os.environ["OMP_NUM_THREADS"] = str(nthreads)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1,'../src/')

In [3]:
from supercellHelpers import *
from buildGRNHelpers import *
from MergeNetworksHelpers import *

In [4]:
#adata = sc.read('../data/microglia.h5ad')

In [5]:
#adata

In [6]:
#adata_merged = supercell_pipeline(adata,
#                                  ngenes=2000,
#                                  npcs=20,
#                                  ncell=500,
#                                  verbose=True)

In [7]:
adata_merged = sc.read('../temp_data/merged_adata.h5ad')

# limit genes to 2000 random genes
# this is to speed up computation for the example
np.random.seed(0)
adata_merged = adata_merged[:,np.random.choice(np.arange(adata_merged.shape[1]),
                                       2000,
                                       replace=False)]

In [8]:
adata_merged

View of AnnData object with n_obs × n_vars = 503 × 2000
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [9]:
all_edges = []
for i in range(10):
    print(i)
    adata_saved = adata_merged.copy()
    
    # -1 all genes
    # 100 neighbors for each gene
    # 10 pcs
    # 0.7 subsample per run
    scing = grnBuilder(adata_merged, -1, 100, 10,0.7,
                      'test','test',12,int(2e9),True)
    scing.subsample_cells()

    scing.filter_genes()
    scing.filter_gene_connectivities()
    scing.build_grn()
    
    all_edges.append(scing.edges)

0
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
1
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
2
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
3
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
4
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
5
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
6
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
7
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
8
building local cluster
Loading data into memory...
Bui

In [10]:
import pickle as pkl
with open('../temp_data/edge.pkl','wb') as f:
    pkl.dump(all_edges, f)
    #all_edges = pkl.load(f)

In [11]:
merger = NetworkMerger(adata_merged,
                    all_edges,
                       0.2,
                    'test',
                    'test',
                    12,
                    int(2e9),
                    True)

In [12]:
merger.preprocess_network_files()
merger.remove_reversed_edges()
merger.remove_cycles()
merger.get_triads()
merger.remove_redundant_edges()

Preprocessing in network files...
Summarizing networks...
Removing bidirectional edges...
Removing cycles...
Getting triads to remove redundant edges...
Removing redundant edges...
Creating client...
Loading data...
Building dask graph...
Closing client...
Removing edges...


In [13]:
merger.edge_df.sort_values(by='importance',
                          ascending=False)

,source,target,importance
90,AC096711.2,ARHGEF16,375.198996
582,MT-ATP6,MT-ND2,358.659181
11405,MT-ND2,MT-ATP6,340.001926
3185,C18orf21,PSMB9,339.932614
510,AC011444.1,AC099343.3,335.266332
...,...,...,...
2780,RMC1,FECH,11.570111
8948,JAK1,STX18,11.538239
2188,FAM214A,ZFYVE21,11.508331
1345,RASGEF1A,CSRP2,11.350153


In [14]:
all_edges

[      importance      source     target
 0   9.993895e+01  AP001107.9      MMGT1
 0   8.064666e+01  AL160254.1     MT1XP1
 0   7.635183e+01        RFC5   ARF4-AS1
 0   6.711786e+01  AL589666.1      HLA-C
 0   6.560508e+01         GSS  PRKCQ-AS1
 ..           ...         ...        ...
 56  1.697211e-16      RILPL1      EXOC1
 69  1.432177e-16  AC091167.6   SLC25A42
 93  1.139837e-16    SLC25A17       GYS1
 48  7.917347e-17       ZC3H4     POLR3B
 90  4.461794e-17      ZNF680   CDC42BPB
 
 [138817 rows x 3 columns],
       importance         source      target
 0   8.206097e+01       C1orf216      ZNF616
 0   7.874011e+01          ZNF92  AC005884.1
 0   6.872901e+01     AL162497.1       MMGT1
 0   6.619958e+01     AC096711.2    ARHGEF16
 0   6.556523e+01          PSMB9      RNF135
 ..           ...            ...         ...
 50  1.366031e-16     AL353796.1  AC009779.2
 68  1.224586e-16          ZNF19      FAM66A
 77  6.541978e-17        SLC35E4       PLOD3
 83  6.394450e-17  ZNF559-ZN